<a href="https://colab.research.google.com/github/CarlHad333/Non_Verbal_Behaviour/blob/main/Text_IEMOCAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#installing some packages that aren't available by default in colab
!pip install neattext
!pip install pytorch_transformers

In [ ]:
#Importing necessary libraries
import numpy as np
import pandas as pd
import os
import re
import neattext.functions as nfx
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import torch
import torch
from transformers import AutoTokenizer, AutoModel
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

In [ ]:
root_path = '/content/gdrive/MyDrive/IEMOCAP_full_release_withoutVideos_sentenceOnly'

df = pd.read_csv(os.path.join(root_path, 'iemocap.csv'))

display(df.head())

,Unnamed: 0,session,method,gender,n_annotators,agreement,emotion,valence,activation,dominance,wav_path,MOCAP_rotated_path,MOCAP_head_path,MOCAP_hand_path,FA_ph_path,FA_st_path,FA_sy_path,FA_wd_path
0,0,1,script,F,3,3,neu,2.5,2.0,2.0,IEMOCAP_full_release/Session1/sentences/wav/Se...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...
1,1,1,script,F,3,2,fru,2.5,2.0,2.5,IEMOCAP_full_release/Session1/sentences/wav/Se...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...
2,2,1,script,F,0,0,xxx,2.0,2.0,2.5,IEMOCAP_full_release/Session1/sentences/wav/Se...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...
3,3,1,script,F,3,2,sur,2.5,3.0,3.0,IEMOCAP_full_release/Session1/sentences/wav/Se...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...
4,4,1,script,F,3,2,neu,2.0,3.0,3.0,IEMOCAP_full_release/Session1/sentences/wav/Se...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/MOCAP_...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...,IEMOCAP_full_release/Session1/sentences/Forced...


In [ ]:
df.columns

Index(['Unnamed: 0', 'session', 'method', 'gender', 'n_annotators',
       'agreement', 'emotion', 'valence', 'activation', 'dominance',
       'wav_path', 'MOCAP_rotated_path', 'MOCAP_head_path', 'MOCAP_hand_path',
       'FA_ph_path', 'FA_st_path', 'FA_sy_path', 'FA_wd_path'],
      dtype='object')

#Data prepping

The purpose of this part is to build a dataset of (text, associated emotions) pairs. This dataset will then be used for emotion recognition.

The function below takes as input the path to a .txt file, and returns a dictionary where the keys are the dialogue sentences, and the values and the emotions associated to the sentence.
\
**How it works:** \
- This is what a réplique in the transcript looks like : "*Ses01M_script02_1_M000 [003.3988-008.5500]: What time is it?  God, this is great, isn't it?  I can't believe it*."
- The fct takes this part '*Ses01M_script02_1_M000*', searches for the corresponding .wav file and the associated emotion (in the DataFrame df)
- It then adds the sentence "*What time is it?  God, this is great, isn't it?  I can't believe it.*" as key, and the value is the emotion found thanks to df.

**NB :** Only data where at least 2 annotators agree are added to our dictionnary

In [ ]:
def create_emotion_dictionary(txt_file):

    # The dictionnary that'll store (sentence: emotion)
    emotion_dict = {}
    #Reading the .txt file
    with open(txt_file, 'r') as file:
        for line in file:
            # Split the line based on ']:'
            parts = line.split(']:', 1)

            # Check if the line has the expected format (to deal with M: F: unidentified speakers)
            if len(parts) == 2:
                # Extract the replique ID from the line (to look for corresponding .wav file)
                replique_id = parts[0].split(' ')[0].strip()
                # Extract the sentence from the line
                sentence = parts[1].strip()

                # pattern to match the .wav file name
                pattern = replique_id.replace('[', '').replace(']', '') + r'.wav'

                # Find the corresponding row in df
                row = df[df['wav_path'].str.contains(pattern)]
                # Check if the row is found in the dataframe
                if not row.empty:
                    # Get the emotion if at least 2 annotators agree on it
                    emotion = row['emotion'].iloc[0]
                    agreement = row['agreement'].iloc[0]
                    if emotion != 'xxx' and agreement >=2:
                    # Add the sentence and emotion to the dictionary
                      emotion_dict[sentence] = emotion
    return emotion_dict

In [ ]:
session_names = ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']
#Function to process all text files in a directory
def process_text_files_in_session(session_directory):
    emotion_dict_all = {}
    for root, dirs, files in os.walk(session_directory):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                emotion_dict = create_emotion_dictionary(file_path)
                emotion_dict_all.update(emotion_dict)
    return emotion_dict_all

root_path = '/content/gdrive/MyDrive/IEMOCAP_full_release_withoutVideos_sentenceOnly'
#process_text_files_in_session(root_path  + '/IEMOCAP_full_release/Session1/dialog/transcriptions')

In [ ]:
# Function to process all sessions
def process_all_sessions(root_path):
    all_emotion_dict = {}
    for session_name in session_names:
        session_directory = os.path.join(root_path, session_name, 'dialog', 'transcriptions')
        session_emotion_dict = process_text_files_in_session(session_directory)
        all_emotion_dict[session_name] = session_emotion_dict
    return all_emotion_dict

In [ ]:
# Example
root_path = '/content/gdrive/MyDrive/IEMOCAP_full_release_withoutVideos_sentenceOnly/IEMOCAP_full_release'
#Dictionnary whose keys are sentences, and whose values are associated emotions
all_sessions_emotion_dict = process_all_sessions(root_path)

In [ ]:
session_dfs = []
for session, data in all_sessions_emotion_dict.items():
    #get session nb (from 'Session1' for instance)
    session_number = session.replace('Session', '')  # Extract session number
    text_data = pd.DataFrame.from_dict(data, orient='index').reset_index()
    text_data.columns = ['sentence', 'emotion']
    text_data['Session'] = session_number  # Add session column
    text_data = text_data[['Session', 'sentence', 'emotion']]  # Reorder columns
    session_dfs.append(text_data)
# Concatenate all session DataFrames into one DataFrame
final_text_data = pd.concat(session_dfs, ignore_index=True)
#Cleaning our text
final_text_data['Clean_Text'] = final_text_data['sentence'].apply(nfx.remove_userhandles)
final_text_data['Clean_Text'] = final_text_data['sentence'].apply(nfx.remove_stopwords)

In [ ]:
final_text_data.head()

,Session,sentence,emotion,Clean_Text
0,1,You okay?,neu,okay?
1,1,"Yeah, I'm just-- yeah",sad,"Yeah, I'm just-- yeah"
2,1,"Just kind of feel numb, you know.",sad,"kind feel numb, know."
3,1,What's happened?,neu,What's happened?
4,1,You didn't even hear?,sad,hear?


In [ ]:
final_text_data.emotion.value_counts()

fru    1716
neu    1531
ang     999
exc     986
sad     958
hap     550
sur      81
fea      35
oth       3
dis       2
Name: emotion, dtype: int64

Not very balanced ...

[Git repo on text emotion recognition](https://github.com/SannketNikam/Emotion-Detection-in-Text/blob/main/notebooks/Emotion%20Detection%20in%20Text.ipynb)


In [ ]:
final_text_data

,Session,sentence,emotion,Clean_Text
0,1,You okay?,neu,okay?
1,1,"Yeah, I'm just-- yeah",sad,"Yeah, I'm just-- yeah"
2,1,"Just kind of feel numb, you know.",sad,"kind feel numb, know."
3,1,What's happened?,neu,What's happened?
4,1,You didn't even hear?,sad,hear?
...,...,...,...,...
6856,5,I don't know. I think this is a pretty good s...,hap,"know. think pretty good spot. mean, look view ..."
6857,5,"Yes, I did notice that. It does look really b...",hap,"Yes, notice that. look beautiful water. Big ol..."
6858,5,"Oh, well look what we've got here.",hap,"Oh, look we've got here."
6859,5,"Well, it isn't champagne.",hap,"Well, champagne."


In [ ]:
selected_emotions = ['neu', 'ang', 'exc', 'sad', 'hap']
final_text_data_filtered = final_text_data[final_text_data['emotion'].isin(selected_emotions)]
final_text_data_filtered.emotion.value_counts()

neu    1531
ang     999
exc     986
sad     958
hap     550
Name: emotion, dtype: int64

In [ ]:
Xfeatures = final_text_data_filtered['Clean_Text']
ylabels = final_text_data_filtered['emotion']

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(Xfeatures,ylabels,test_size=0.3,random_state=42)

[Count Vectorizer explained](https://spotintelligence.com/2023/05/17/countvectorizer/#:~:text=CountVectorizer%20is%20a%20text%20preprocessing,machine%20learning%20library%20in%20Python.)

- **Used model:** Logistic regression
- **Feature:** CountVectorizer (cf. intro  above)

In [ ]:
pipe_lr = Pipeline(steps=[('cv',CountVectorizer()),('lr',LogisticRegression(max_iter = 500))])
pipe_lr.fit(x_train,y_train)
print('Accuracy: ')
pipe_lr.score(x_test,y_test)

Accuracy: 


0.5583554376657824

- **Used model:** Logistic regression
- **Feature:** Tf-Idf Vectorizer

In [ ]:
pipe_lr_tfidf = Pipeline(steps=[
    ('tfidf', TfidfVectorizer()),
    ('lr', LogisticRegression(max_iter=1000))
])

# Fit the pipeline on training data
pipe_lr_tfidf.fit(x_train, y_train)

# Evaluate the model on test data
accuracy = pipe_lr_tfidf.score(x_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.5610079575596817


**Logistic regression but this time with a more balanced dataset**

*Balancing the dataset*

In [ ]:
selected_emotions = ['neu', 'ang', 'exc', 'sad', 'hap']
final_text_data_filtered = final_text_data[final_text_data['emotion'].isin(selected_emotions)].copy()

# Replace 'exc' with 'hap'
final_text_data_filtered.loc[final_text_data_filtered['emotion'] == 'exc', 'emotion'] = 'hap'

# Display the value counts after replacement
print(final_text_data_filtered['emotion'].value_counts())
balanced_data = final_text_data_filtered.groupby('emotion').head(963)
LE = LabelEncoder()
balanced_data['label'] = LE.fit_transform(balanced_data['emotion'])
# Display the value counts to ensure balance
print(balanced_data['emotion'].value_counts())

hap    1536
neu    1531
ang     999
sad     958
Name: emotion, dtype: int64
neu    963
ang    963
hap    963
sad    958
Name: emotion, dtype: int64


<ipython-input-19-9bd96d84771a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balanced_data['label'] = LE.fit_transform(balanced_data['emotion'])


In [ ]:
balanced_data.head()

,Session,sentence,emotion,Clean_Text,label
0,1,You okay?,neu,okay?,2
1,1,"Yeah, I'm just-- yeah",sad,"Yeah, I'm just-- yeah",3
2,1,"Just kind of feel numb, you know.",sad,"kind feel numb, know.",3
3,1,What's happened?,neu,What's happened?,2
4,1,You didn't even hear?,sad,hear?,3


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(balanced_data['Clean_Text'], balanced_data['label'], test_size=0.2, random_state=42)


**Star of the day : logistic regression**

In [ ]:
pipe_lr = Pipeline(steps=[('cv',CountVectorizer()),('lr',LogisticRegression(max_iter = 500))])
pipe_lr.fit(X_train,Y_train)
print('Accuracy: ')
pipe_lr.score(X_test,Y_test)

Accuracy: 


0.6116883116883117

In [ ]:
pipe_lr_tfidf = Pipeline(steps=[
    ('tfidf', TfidfVectorizer()),
    ('lr', LogisticRegression(max_iter=1000))
])

# Fit the pipeline on training data
pipe_lr_tfidf.fit(X_train, Y_train)

# Evaluate the model on test data
accuracy = pipe_lr_tfidf.score(X_test, Y_test)
print("Accuracy:", accuracy)

Accuracy: 0.6155844155844156


**XGBoost**

In [ ]:
pipe_xgb_cv = Pipeline(steps=[('cv', CountVectorizer()), ('xgb', XGBClassifier())])
pipe_xgb_cv.fit(X_train,Y_train)
print('Accuracy: ')
pipe_xgb_cv.score(X_test,Y_test)

Accuracy: 


0.6012987012987013

In [ ]:
pipe_xgb_tfidf = Pipeline(steps=[ ('tfidf', TfidfVectorizer()), ('xgb', XGBClassifier())])
pipe_xgb_tfidf.fit(X_train,Y_train)
print('Accuracy: ')
pipe_xgb_tfidf.score(X_test,Y_test)

Accuracy: 


0.5883116883116883

#Attempt to use BERT for feature extraction from text data

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states=True)

NameError: name 'BertTokenizer' is not defined

In [ ]:
#converting text labels (emotions) into numerical form
LE = LabelEncoder()
final_text_data_filtered['label'] = LE.fit_transform(final_text_data_filtered['emotion'])

In [ ]:
final_text_data_filtered.head()

In [ ]:
from transformers import BertTokenizer, BertModel


In [ ]:
len(final_text_data_filtered['Clean_Text'].tolist())

In [ ]:
filtered_list = final_text_data_filtered['Clean_Text'].tolist()
first_2000_elements = filtered_list[:3000]

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize data
data_tokenized = tokenizer(filtered_list, padding=True, truncation=True, return_tensors='pt')

# Load pre-trained BERT model
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model.eval()  # Set the model to evaluation mode

# Extract BERT embeddings for training data
with torch.no_grad():
    outputs = bert_model(**data_tokenized)
    pooled_output = outputs.pooler_output  # Extract pooled representation

# Convert pooled output to numpy array
bert_data_features = pooled_output.numpy()

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(bert_data_features, final_text_data_filtered['label'], test_size=0.3, random_state=42)

In [ ]:
print(X_train[0].shape)  # Check shape of first training example


In [ ]:
X_train = X_train.reshape((X_train.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))

X_train.shape

In [ ]:
Y_train.shape

In [ ]:
X_train =(X_train - np.mean(X_train))/np.std(X_train)

In [ ]:
# Reshape the features into a 2D array
# Train the logistic regression model
classifier = LogisticRegression(max_iter = 500)
classifier.fit(X_train,y= Y_train)

In [ ]:
#Compute accuracy with logistic regression
X_test =(X_test - np.mean(X_test))/np.std(X_test)
from sklearn.metrics import accuracy_score
predictions = classifier.predict(X_test)
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy:", accuracy)

In [ ]:
# Define and train your CNN model
import torch.nn as nn
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertModel
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import torch.nn as nn
import torch.optim as optim
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=X_train_features.shape[1], out_channels=128, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(64 * 5, 64)
        self.fc2 = nn.Linear(64, 5)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert features and labels to PyTorch tensors
X_train_tensor = torch.tensor(X_train_features, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_features, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.int64)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.int64)

# Create DataLoader for training and testing sets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Train the model
epochs = 2
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.permute(0, 2, 1))  # Permute dimensions for CNN input
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs.permute(0, 2, 1))  # Permute dimensions for CNN input
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy}")


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_classifier(nn.Module):
    def __init__(self, num_classes, input_channels=1):
        super(CNN_classifier, self).__init__()
        self.num_classes = num_classes

        # Convolutional layers
        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)

        # Maxpool layers
        self.maxpool = nn.MaxPool1d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(128 * (768 // 8), 256)  # Adjusted based on maxpooling
        self.fc2 = nn.Linear(256, num_classes)

        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = x.unsqueeze(0)
        # Convolutional layers with ReLU activation and maxpooling
        x = F.relu(self.conv1(x.unsqueeze(1)))  # Add unsqueeze to add a channel dimension
        print(x.shape)
        x = self.maxpool(x)
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv3(x))
        x = self.maxpool(x)

        # Flatten the output for fully connected layers
        x = x.view(x.size(0), -1)

        # Fully connected layers with dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        # Apply softmax for multi-class classification
        x = F.softmax(x, dim=1)

        return x


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_classifier(nn.Module):
    def __init__(self, num_classes, input_channels=1):
        super(CNN_classifier, self).__init__()
        self.num_classes = num_classes

        # Convolutional layers
        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)

        # Maxpool layers
        self.maxpool = nn.MaxPool1d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 96, 256)  # 128 * 192 is the output size of conv3
        self.fc2 = nn.Linear(256, num_classes)

        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        # Convolutional layers with ReLU activation and maxpooling
        x = F.relu(self.conv1(x))
        x = self.maxpool(x)
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv3(x))
        x = self.maxpool(x)

        # Flatten the output for fully connected layers
        x = x.view(x.size(0), -1)

        # Fully connected layers with dropout

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        # Apply softmax for multi-class classification
        x = F.softmax(x, dim=1)

        return x


In [ ]:
from torchsummary import summary
model = CNN_classifier(5, input_channels=1)
summary(model, (1,768))